# 🚀 AlphaKhulnasoft: Competitive AI Code Repair

Welcome to the **AlphaKhulnasoft** demo! This notebook allows you to run the full flow-engineered code repair pipeline in Google Colab.

### 💡 What is AlphaKhulnasoft?
It is a "System 2" agentic framework that doesn't just generate code once. It **Analyze → Generates → Tests → Debugs → Repairs** until it passes or hits a limit.

**Note:** To run this demo, you will need an OpenAI, Anthropic, or Google Cloud (Vertex AI) API key.

## 🛠️ Step 1: Environment Setup

First, we install `uv` and setup our project environment.

In [ ]:
# Install uv
!curl -LsSf https://astral.sh/uv/install.sh | sh
import os

os.environ["PATH"] = f"{os.environ['HOME']}/.cargo/bin:{os.environ['PATH']}"

# Clone the repository (if not already present)
if not os.path.exists("AlphaKhulnasoft"):
    !git clone https://github.com/KhulnaSoft/AlphaKhulnasoft.git

%cd AlphaKhulnasoft

# Install dependencies
!uv sync

## 🔑 Step 2: Configure Authentication

Choose your provider below.

### Option A: OpenAI / Anthropic / Hugging Face
Set your API keys manually or use Colab Secrets.

In [ ]:
import getpass
import os

# OpenAI
if "OPENAI_API_KEY" not in os.environ:
    val = getpass.getpass("Enter your OpenAI API Key (leave empty to skip): ")
    if val:
        os.environ["OPENAI_API_KEY"] = val

# Anthropic
if "ANTHROPIC_API_KEY" not in os.environ:
    val = getpass.getpass("Enter your Anthropic API Key (leave empty to skip): ")
    if val:
        os.environ["ANTHROPIC_API_KEY"] = val

# Hugging Face
if "HF_TOKEN" not in os.environ:
    val = getpass.getpass("Enter your Hugging Face Token (leave empty to skip): ")
    if val:
        os.environ["HF_TOKEN"] = val

print("✅ Manual keys configured.")

### Option B: Google Cloud Vertex AI
Run the cell below to authenticate with your Google Account and set your Project ID.

In [ ]:
from google.colab import auth

auth.authenticate_user()

PROJECT_ID = "your-project-id"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

os.environ["VERTEX_PROJECT"] = PROJECT_ID
os.environ["VERTEX_LOCATION"] = LOCATION

print(f"✅ Vertex AI configured for project: {PROJECT_ID}")

## 🌋 Step 3: Load or Generate Problems

You can either bootstrap a "Golden Dataset" with an LLM or fetch a benchmark directly from **Hugging Face Hub**.

In [ ]:
# Choice A: Generate with LLM
!uv run python -m alphakhulnasoft.dataset_gen --count 3

# Choice B: Fetch from Hugging Face (e.g., HumanEval)
import json
from alphakhulnasoft.data_loader import DataLoader

loader = DataLoader()
hf_problems = loader.load_from_hf("openai/humaneval", split="test")
print(f"📥 Loaded {len(hf_problems)} problems from Hugging Face.")

## ⚔️ Step 4: Run the Gauntlet (The Benchmark)

Now we run the **AlphaRepairAgent** against the target problems.

In [ ]:
!uv run python -m alphakhulnasoft.benchmark data/hard_mode.jsonl

## 📊 Step 5: Visualize and Publish

Finally, we generate charts and optionally publish results to the **Hugging Face Hub**.

In [ ]:
import glob
import IPython
from alphakhulnasoft.publisher import HFPublisher

# Find the latest results file
results_files = glob.glob("results_*.json")
if results_files:
    latest_results = max(results_files)
    !uv run python -m alphakhulnasoft.visualizer {latest_results}

    # Display graphs
    print("\n--- 📈 Repair Trajectory ---")
    IPython.display.display(IPython.display.Image("repair_curve.png"))

    # Optional: Publish to Hugging Face
    # repo_id = "your-username/your-repo-name"
    # if os.environ.get("HF_TOKEN"):
    #     publisher = HFPublisher(repo_id)
    #     publisher.publish_results(latest_results)
else:
    print("❌ No results found.")